In [33]:
%matplotlib inline
import cv2, os, ast, time, math, shutil, sys, glob 
from datetime import datetime
from PIL import Image
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
tqdm.pandas() 
from joblib import Parallel, delayed

import torch
import torchvision
from torchvision.ops import box_iou
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
print(torchvision.__version__)

0.13.0


In [34]:
def coco2yolo(image_height, image_width, bboxes):
    """
    coco => [xmin, ymin, w, h]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    for bbox in bboxes:
        if bbox[0] + bbox[2] >= image_width:
            bbox[2] = image_width - bbox[0] - 1
        if bbox[1] + bbox[3] >= image_height:
            bbox[3] = image_height - bbox[1] - 1
    
    # normolizinig
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]/ image_height
    
    # converstion (xmin, ymin) => (xmid, ymid)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]/2
    
    return bboxes

In [37]:
%cd ..
cwd = os.getcwd()
%cd ..

h:\Python\ob
h:\Python


In [38]:
base_dir = cwd
train_csv = os.path.join(base_dir, "train.csv")
train_df = pd.read_csv(train_csv)
train_df["img_path"] = os.path.join(base_dir, "train_images") + "/video_" + train_df.video_id.astype(str) + "/" + train_df.video_frame.astype(str) + ".jpg"
train_df["annotations"] = train_df["annotations"].apply(eval)
train_df["a_count"] = train_df["annotations"].apply(len)
train_df = train_df.drop(columns=['video_id', 'sequence', 'video_frame', 'sequence_frame'])
train_df_positive = train_df[train_df['a_count'] != 0]
train_df_positive= train_df_positive.reset_index(drop=True)
print('shape of train data with annotations:', train_df_positive.shape)

train_df_ratio = (train_df_positive.set_index('image_id').explode('annotations').
                  apply(lambda row: pd.Series(row['annotations']), axis=1).reset_index())
train_df_ratio['aspect_ratio'] = train_df_ratio['height']/train_df_ratio['width']


train_df_p, val_df_p = train_test_split(train_df_positive, test_size=0.1, random_state=0)
print('shape of training data:', train_df_p.shape)
print('shape of validation data:', val_df_p.shape)
print('Min number of bboxs per image:', min(train_df_positive.a_count))
print('Max number of bboxs per image:', max(train_df_positive.a_count))
print('Max width of all bboxs:', max(train_df_ratio.width))
print('Max height of all bboxs:', max(train_df_ratio.height))
print('Min width of all bboxs:', min(train_df_ratio.width))
print('Min height of all bboxs:', min(train_df_ratio.height))

print('{:.2%} of all aspect ratios greater than 2'.
      format(train_df_ratio[train_df_ratio['aspect_ratio'] > 1.5].shape[0]/train_df_ratio.shape[0]))

print('{:.2%} of all aspect ratios less than 0.25'.
      format(train_df_ratio[train_df_ratio['aspect_ratio'] < 0.5].shape[0]/train_df_ratio.shape[0]))

shape of train data with annotations: (4919, 4)
shape of training data: (4427, 4)
shape of validation data: (492, 4)
Min number of bboxs per image: 1
Max number of bboxs per image: 18
Max width of all bboxs: 243
Max height of all bboxs: 222
Min width of all bboxs: 17
Min height of all bboxs: 13
1.47% of all aspect ratios greater than 2
1.15% of all aspect ratios less than 0.25


In [ ]:
def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

train_df_p['bboxes'] = train_df.annotations.apply(get_bbox)
val_df_p['bboxes'] = train_df.annotations.apply(get_bbox)

In [ ]:
%cd train_models/
cwd = os.getcwd()

IMAGE_DIR = "/images"
LABEL_DIR = "/labels"

os.makedirs(cwd + '/images')
os.makedirs(cwd + '/labels')

os.makedirs(cwd + '/images/train')
os.makedirs(cwd + '/labels/train')
os.makedirs(cwd + '/images/val')
os.makedirs(cwd + '/labels/val')


In [ ]:
def writeLabels(bboxes, destfile, image_width = 1280, image_height = 720):
    bboxes_coco  = np.array(bboxes).astype(np.float32).copy()
    num_bbox     = len(bboxes_coco)
    names        = ['cots'] * num_bbox
    labels       = [0] * num_bbox
    with open(destfile, 'w') as f:
        if num_bbox<1:
            annot = ''
            f.write(annot)
        else:
            bboxes_yolo  = coco2yolo(image_height, image_width, bboxes_coco)
            for bbox_idx in range(len(bboxes_yolo)):
                annot = [str(labels[bbox_idx])]+ list(bboxes_yolo[bbox_idx].astype(str))+(['\n'] if num_bbox!=(bbox_idx+1) else [''])
                annot = ' '.join(annot)
                annot = annot.strip(' ')
                f.write(annot)
    return None 

In [ ]:
for row_idx in tqdm(range(len(train_df_p))):
    row = train_df_p.iloc[row_idx]
    shutil.copyfile(row.img_path, f'{cwd}/images/train/{row.image_id}.jpg')
    writeLabels(row.bboxes, f'{cwd}/labels/train/{row.image_id}.txt')

for row_idx in tqdm(range(len(val_df_p))):
    row = val_df_p.iloc[row_idx]
    shutil.copyfile(row.img_path, f'{cwd}/images/val/{row.image_id}.jpg')
    writeLabels(row.bboxes, f'{cwd}/labels/val/{row.image_id}.txt')

In [ ]:
file_path = []
cots_dir = os.path.join(base_dir, 'train_models/YOLO/yolov7/cots')
os.chdir(cots_dir)
for file in glob.glob(os.path.join(cots_dir, 'images/train/*.jpg')):
    file_path.append(file)

textfile = open("./train.txt", "w")
for element in file_path:
    textfile.write(element + "\n")
textfile.close()


file_path = []
for file in glob.glob(os.path.join(cots_dir, 'images/val/*.jpg')):
    file_path.append(file)

textfile = open("./val.txt", "w")
for element in file_path:
    textfile.write(element + "\n")
textfile.close()
